In [1]:
import string

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

import nltk
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('vader_lexicon')

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.sentiment import SentimentIntensityAnalyzer

from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shrutirao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shrutirao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/shrutirao/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [103]:
df = pd.read_csv('data/lab42_cleaned_data.csv', index_col=0)

In [110]:
def apply_token(row):
    
    stop_words = set(stopwords.words("english"))
    words_in_sent = word_tokenize(row)

    filtered_list = []
    
    for word in words_in_sent:
        if word.casefold() not in stop_words:
            filtered_list.append(word.lower())
    
    return filtered_list


def remove_unecessary_tokens(tokens):
    cleaned_tokens = list(filter(lambda x: len(x) > 2, tokens))

    return cleaned_tokens


def emotion_specific_cleanup(row):
    filler_words = ["bit", "little", "feeling",
                   "look", "like", "assignment", "exam",
                    "feel", "feels", "feeling", "looks", "around",
                    "wanted", "last", "night", "say", "building", 
                    "able", "let", "able", "also", "space",
                    "study", "studying"
                   ]
    
    cleaned_list = list(filter(lambda x: x not in filler_words, row))
    return cleaned_list  


def comfort_specific_cleanup(row):
    filler_words = ["bit", "little", "feeling",
                   "look", "like", "assignment", "exam",
                    "feel", "feels", "feeling", "looks", "around",
                    "wanted", "last", "night", "say", "building", 
                    "able", "let", "able", "also", "space",
                    "study", "studying", "many", "make"
                   ]
    
    cleaned_list = list(filter(lambda x: x not in filler_words, row))
    return cleaned_list    

# Emotion

In [20]:
df_ground_floor = df.loc[df['floor'] == "Ground floor"][['ground-floor', 'emotion']].dropna()
df_1_floor = df.loc[df['floor'] == "1st Floor"][['1-floor', 'emotion']].dropna()
df_2_floor = df.loc[df['floor'] == "2nd Floor"][['2-floor', 'emotion']].dropna()
df_3_floor = df.loc[df['floor'] == "3rd Floor"][['3-floor', 'emotion']].dropna()

In [94]:
floors = [df_ground_floor, df_1_floor, df_2_floor, df_3_floor]
names = ["ground-floor", "1-floor", "2-floor", "3-floor"]

emo_df = []

for floor, name in zip(floors, names):
    floor['tokens'] = floor.emotion.apply(apply_token)
    floor['tokens'] = floor.tokens.apply(remove_unecessary_tokens)
    floor['tokens'] = floor.tokens.apply(emotion_specific_cleanup)
    emo_df.append(pd.DataFrame(floor.groupby([name])['tokens'].sum().reset_index()))

In [96]:
for space, emos in zip(emo_df[0]["ground-floor"], emo_df[0].tokens):
    print(space, " : ", sorted(emos), "\n")

Green chairs by the entrance  :  ['chill', 'nice', 'pretty', 'quiet', 'relaxed'] 

Library learning room  :  ['aesthetic', 'calm', 'calm', 'cool', 'energising', 'exams', 'happy', 'motivated', 'normal', 'room', 'stressed'] 

Next to the lockers  :  ['acceptable', 'acoustics', 'als', 'atmosphere', 'atmosphere', 'background', 'calm', 'exposed', 'good', 'level', 'makes', 'near', 'neutral', 'noise', 'noises', 'others', 'overall', 'personally', 'relaxed', 'sad', 'work'] 

Round tables by the three plants (across wooden staircase)  :  ['atmosphere', 'atmosphere', 'big', 'calm', 'calm', 'calm', 'calm', 'cold', 'collected', 'concentration', 'cramped', 'fatigue', 'good', 'happy', 'majestic', 'mildly', 'moment', 'motivated', 'neutral', 'neutral', 'open', 'relaxed', 'rest', 'safe', 'stable', 'view'] 

Study corner next to the plant wall  :  ['cold', 'neutral', 'normal', 'pretty', 'relaxing', 'stressed'] 

Unsure  :  ['comfortable', 'happy', 'neutral', 'tired', 'tired'] 



In [97]:
for space, emos in zip(emo_df[1]["1-floor"], emo_df[1].tokens):
    print(space, " : ", sorted(emos), "\n")

Green group study tables (between a row of plants and railing)  :  ['ago', 'annoying', 'apart', 'booting', 'bored', 'calm', 'calm', 'calm', 'calm', 'calm', 'ceiling', 'chill', 'class', 'cold', 'comfortable', 'comfortable', 'comfortable', 'comfy', 'couple', 'cozy', 'cozy', 'dark', 'depends', 'directly', 'enjoyable', 'face', 'fact', 'fine', 'floors', 'focussed', 'good', 'good', 'good', 'got', 'guess', 'happy', 'hours', 'hungry', 'kindoff', 'light', 'mood', 'mood', 'mood', "n't", 'neutral', 'nice', 'nice', 'nice', 'normal', 'otherwise', 'peaceful', 'place', 'positive', 'quite', 'really', 'relaxed', 'relaxed', 'sad', 'shining', 'since', 'slowly', 'sometimes', 'somewhat', 'tired', 'tired', 'top', 'use', 'well', 'whether'] 

Library learning room  :  ['calm', 'classrooms', 'comfortable', 'comfortable', 'comfortable.make', 'concentration', 'due', 'easily', 'focused', 'getting', 'green', 'hear', 'impact', 'negative', 'nice', 'nice', 'noisy', 'offices/flexdesks', 'peaceful', 'person', 'quiet', 

In [99]:
for space, emos in zip(emo_df[2]["2-floor"], emo_df[2].tokens):
    print(space, " : ", sorted(emos), "\n")

Group tables by the stairs  :  ['lighting', 'need', 'yes'] 

Library learning room  :  ['artificial', 'automatically', 'blinds', 'calm', 'calm', 'calm', 'calm', 'chill', 'closed', 'dark', 'day', 'day', 'focus', 'focused', 'frustrated', 'general', 'good', 'lighting', 'location', 'loving', 'makes', 'much', 'natural', 'open-space', 'peace', 'relaxed', 'resources', 'silence', 'sunlight', 'sunny', 'tired', 'used'] 

Unsure  :  ['3rd', 'areas', 'bridge', 'coffee', 'floor', 'lifeless', 'main', 'putside', 'shared', 'terrible'] 



In [100]:
for space, emos in zip(emo_df[3]["3-floor"], emo_df[3].tokens):
    print(space, " : ", sorted(emos), "\n")

Near the printer  :  ['day', 'tiredafter'] 

Open lounge area  :  ['anxious', 'neutral', 'open', 'pleasant'] 

Round tables by the coffee machine  :  ['happy'] 

Unsure  :  ['good', 'nice', 'sterile'] 



# Comfort

In [111]:
df_ground_floor = df.loc[df['floor'] == "Ground floor"][['ground-floor', 'comfort']].dropna()
df_1_floor = df.loc[df['floor'] == "1st Floor"][['1-floor', 'comfort']].dropna()
df_2_floor = df.loc[df['floor'] == "2nd Floor"][['2-floor', 'comfort']].dropna()
df_3_floor = df.loc[df['floor'] == "3rd Floor"][['3-floor', 'comfort']].dropna()

In [112]:
floors = [df_ground_floor, df_1_floor, df_2_floor, df_3_floor]
names = ["ground-floor", "1-floor", "2-floor", "3-floor"]

comfort_df = []

for floor, name in zip(floors, names):
    floor['tokens'] = floor.comfort.apply(apply_token)
    floor['tokens'] = floor.tokens.apply(remove_unecessary_tokens)
    floor['tokens'] = floor.tokens.apply(emotion_specific_cleanup)
    comfort_df.append(pd.DataFrame(floor.groupby([name])['tokens'].sum().reset_index()))

In [113]:
for space, comf in zip(comfort_df[0]["ground-floor"], comfort_df[0].tokens):
    print(space, " : ", sorted(comf), "\n")

Green chairs by the entrance  :  ['comforting', 'especially', 'everything', 'front', 'garden', 'nice', 'nice', 'spacious', 'sure', 'terrace'] 

Library learning room  :  ['amount', 'cold', 'cold', 'colours', 'comfortable', 'comfortable', 'comfortably', 'daylight', 'great', 'lot', 'make', 'makes', 'many', 'nice', 'noises', 'normal', 'open', 'really', 'right', 'room', 'sunlight', 'usually', 'warm', 'warm', 'window'] 

Next to the lockers  :  ['air', 'annoying', 'comfort', 'comfortable', 'comfortable', 'couch', 'dark', 'door', 'every', 'focused', 'full', 'good', 'good', 'hard', 'leaving', 'life', 'lighting', 'maybe', 'minute', 'noisy', 'quality', 'quite', 'shadows', 'shawl', 'sitting', 'slamming', 'soft', 'sometimes', 'sound', 'temperature', 'thing', 'though', 'toilet', 'warm'] 

Round tables by the three plants (across wooden staircase)  :  ['background', 'blinded', 'breaks', 'close', 'cold', 'cold', 'cold', 'cold', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'concerning'

In [115]:
for space, comf in zip(comfort_df[1]["1-floor"], comfort_df[1].tokens):
    print(space, " : ", sorted(comf), "\n")

Green group study tables (between a row of plants and railing)  :  ['actually', 'air', 'alright', 'annoying', 'background', 'better', 'chairs', 'chairs', 'class', 'classes', 'cold', 'cold', 'color', 'comfertable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'dark', 'downside', 'end', 'ended', 'even', 'face', 'feet', 'find', 'floor', 'good', 'good', 'good', 'great', 'greenery', 'light', 'light', 'light', 'light', 'lot', 'loud', 'loud', 'mind', 'much', 'much', "n't", 'natural', 'nice', 'nice', 'nice', 'noise', 'noise', 'noise', 'noise', 'noise', 'noisy', 'noisy', 'office', 'others', 'people', 'perfect', 'perfect', 'place', 'plants', 'pretty', 'pretty', 'pretty', 'quality', 'quite', 'quite', 'really', 'right', 'room', 'said', 'seats', 'shining', 'short', 'since', 'smaller', 'spaces', 'taste', 'temperature', 'temperature', 'though', 'times',

In [116]:
for space, comf in zip(comfort_df[2]["2-floor"], comfort_df[2].tokens):
    print(space, " : ", sorted(comf), "\n")

Group tables by the stairs  :  ['heating', 'lighting', 'need', 'need', 'yes'] 

Library learning room  :  ['air', 'air', 'area', 'aspects', 'automatic', 'becomes', 'besides', 'better', 'blinds', 'blinds', 'blinds', 'blockers', 'blocking', 'closed', 'closed', 'closing', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'comfortable', 'could', 'curtains', 'dark', 'dark', 'dark', 'enough', 'facing', 'get', 'gloomy', 'good', 'good', 'good', 'helps', 'inside', 'irregularity', 'light', 'light', 'lit', 'make', 'making', 'natural', 'nice', 'nicely', 'noises', 'often', 'open/staircase', 'please', 'point', 'pretty', 'pretty', 'pretty', 'quality', 'quality', 'room', 'room', 'sad', 'screen', 'slightly', 'stop', 'sun', 'sunny', 'temperature', 'though', 'uncomfortable', 'whole', 'window', 'window', 'worse'] 

Unsure  :  ['alright', 'everything'] 



In [117]:
for space, comf in zip(comfort_df[3]["3-floor"], comfort_df[3].tokens):
    print(space, " : ", sorted(comf), "\n")

Near the printer  :  ['fine', 'noisy', 'sometimes'] 

Open lounge area  :  ['always', 'cold', 'comfortable', 'dont', 'fine', 'fine', 'light', 'noise', 'open', 'open', 'pretty', 'temperature'] 

Round tables by the coffee machine  :  ['comfortable'] 

Unsure  :  [] 

